# 07 Integration (Part 3)
See *Computational Physics* (Landau, Páez, Bordeianu), Chapter 5.7 - 5.22

These slides include material from  *Computational Physics. eTextBook Python 3rd Edition.* Copyright © 2012 Landau, Rubin, Páez. Used under the Creative-Commons Attribution-NonCommerical-ShareAlike 3.0 Unported License.


# Trapezoid rule 
Approximate the area by the sum of the trapezoid areas under the curve.... 
![Trapezoid integration](images/CompPhys2012_trapezoid.png)

... which is what our `integrate_simple()` function already does!


In [ ]:
trapezoid = integrate_simple

Formally, the *trapezoid rule* consists of the sum

\begin{gather}
\begin{split}
I = h\left[ \frac{f(a) + f(x_1)}{2} + \frac{f(x_1) + f(x_2)}{2} + \frac{f(x_2) + f(x_3)}{2} + \\
    + \cdots + \frac{f(x_{N-1}) + f(b)}{2} \right]
\end{split}\\
  = h\left[ \frac{f(a)}{2} + f(x_1) + f(x_2) + \cdots + f(x_{N-1}) + \frac{f(b)}{2} \right]
\end{gather}

## General form of an integration algorithm

$$
\int_a^b \!f(x)\, dx \approx \sum_{i=1}^N w_i\ f(x_i)
$$

with weights $w_i$ and integration points $x_i$ that depend on the integration algorithm.

For the *trapezoid rule*, the weights are simply $\{\frac{1}{2}, \frac{1}{2}\} \times h$ for two adjacent points and for the whole interval with $N$ points $\{\frac{1}{2}, 1, 1, \dots, 1, \frac{1}{2}\} \times h$.

## Simpson's rule
Interpolate parabolas:

![Simpson's rule](./images/CompPhys2012_Simpson.png)

For each interval $i$, approximate $f(x)$ as

$$
f(x) \approx \alpha_i x^2 + \beta_i x + \gamma_i
$$

and the area under each parabola (equally spaced intervals of width $h$) is

$$
\int_{x_i}^{x_i + h} (\alpha_i x^2 + \beta_i x + \gamma_i) dx = \left[\frac{\alpha_i x^3}{3} + \frac{\beta_i x^2}{2} + \gamma_i x\right]_{x_i}^{x_i + h}
$$

What should the coefficients $\alpha_i$, $\beta_i$, $\gamma_i$ be? Express them through function values.

For example for the interval $[-1, 1]$:

$$
\int_{-1}^{1} (\alpha x^2 + \beta x + \gamma) dx = \frac{2\alpha}{3} + 2\gamma
$$

and select function values at $-1, 0, 1$:

$$
f(-1) = \alpha - \beta + \gamma; \quad f(0) = \gamma; \quad f(1) = \alpha + \beta + \gamma
$$

(exact because we can use three points to determine a parabola!)

Thus
$$
\alpha = \frac{f(1) + f(-1)}{2} - f(0); \quad \beta = \frac{f(1) - f(-1)}{2};\quad \gamma = f(0)
$$

and the integral is
$$
\int_{-1}^{1} (\alpha x^2 + \beta x + \gamma) dx = \frac{1}{3}f(-1) + \frac{4}{3} f(0) + \frac{1}{3} f(1)
$$
(weights $\{\frac{1}{3}, \frac{4}{3}, \frac{1}{3}\} \times 1$ for three adjacent points)

Generalize to 3 points $x_i - h$, $x_i$, and $x_i + h$:

\begin{gather}
\begin{split}
\int_{x_i - h}^{x_i + h} f(x) \,dx = \int_{x_i - h}^{x_i} f(x) \,dx + \int_{x_i}^{x_i + h} f(x) \,dx = \dots \approx \\
\approx \frac{h}{3} f_{i-1} + \frac{4h}{3} f_i + \frac{h}{3} f_{i+1}
\end{split}
\end{gather}

Simpson's rule: integrate over *pairs of intervals* so we need an *even number of intervals*, i.e., $N$ to be *odd*.


Sum over all intervals, count every interval point twice except the very first and last end point:

\begin{gather}
\begin{split}
\int_a^b f(x) \,dx \approx \frac{h}{3} f_1 + \frac{4h}{3} f_2 + \frac{2h}{3} f_3 + \frac{4h}{3} f_4 + \\
 + \dots + \frac{4h}{3}f_{N-1} +  \frac{h}{3} f_N
\end{split}
\end{gather}

so the weights are for the full interval with $N$ points $\{\frac{1}{3}, \frac{4}{3}, \frac{2}{3}, \frac{4}{3}, \dots, \frac{4}{3}, \frac{1}{3}\} \times h \equiv \{1, 4, 2, 4, 2, \dots, 4, 1\} \times \frac{1}{3} \times h$.



The sum of weights has to be
$$
\sum_{i=1}^N w_i = N - 1
$$

(Note that one often makes $h$ part of the weights and then the sum should equal $(N-1)h$.)

### Implement Simpson's rule in a Python function 

In [ ]:
def simpson(f, a, b, N):
    if N % 2 == 0:
        raise ValueError("N must be odd")
    h = (b - a)/(N - 1)
    Ih = -f(a) + f(b)        # sum 2, 4; 2, 4, pairs: need to make first contrib 1
    for i in range(0, N-1, 2):
        xi = a + i*h
        Ih += 2*f(xi) + 4*f(xi + h)
    Ih *= h/3
    return Ih

In [ ]:
simpson(np.cos, -np.pi/2, np.pi/2, 11)

In [ ]:
%timeit simpson(np.cos, -np.pi/2, np.pi/2, 101)

$$
\int_{-10\pi}^{\pi/2} \cos x\, dx = 1
$$

In [ ]:
simpson(np.cos, -10*np.pi, np.pi/2, 11)

In [ ]:
simpson(np.cos, -10*np.pi, np.pi/2, 101)

#### Simpson's rule without loops (ALL HAIL NUMPY!) 

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("8AOfbnGkuGc", autoplay=1)

(Copyright © Twentieth Century Fox)

As numpy-based function: basic idea: calculate
```python
sum(weights * functionvalues)
```

Main problem: get the weights right. Develop the algorithm step-by-step interactively:
* test `N = 11`
* weights `1, 4, 2, 4, 2, ..., 2, 4, 1` (`N` weights in total)
* divide by 3 and multiply with `h` at the end

Use *assigning to numpy slices* for an efficient algorithm (to get `1, 4, 2, 4, 2, ..., 2, 4, 1`):

In [ ]:
import numpy as np

Now use these weights to program `simpson_np(f, a, b, N)`, which uses `sum(weights * functionvalues)`:

In [ ]:
import numpy as np

def simpson_np(f, a, b, N):
    if N % 2 == 0:
        raise ValueError("N must be odd")
    # implement ...

In [ ]:
%timeit simpson_np(np.cos, -np.pi/2, np.pi/2, 101)

## Uniform step integration rules


| Name          |     Degree    |  Elementary weights                                  |
|---------------|:-------------:|------------------------------------------------------|
| Trapezoid     | 1             | $\{1, 1\}\times \frac{1}{2} \times h$                |
| Simpson's     | 2             | $\{1, 4, 1\}\times \frac{1}{3} \times h$             |
| $\frac{3}{8}$ | 3             | $\{1, 3, 3, 1\}\times \frac{3}{8} \times h$          |
| Milne         | 4             | $\{14, 64, 24, 64, 14\}\times \frac{1}{45} \times h$ |

In order to derive the *weights* for $N$ points one needs to overlap the elementary weights so that the sub-interval end points are counted twice *except* the very first ($a$) and the very last $(b$) points.

## Using scipy.integrate
In general, make use of library routines. [scipy](http://scipy.org) is a collection of numpy-based scientific algorithms.

In [ ]:
import scipy.integrate

In [ ]:
X = np.linspace(-np.pi/2, np.pi/2, 10*4 + 1)
scipy.integrate.simps(np.cos(X), X)

In [ ]:
%timeit scipy.integrate.simps(np.cos(X), X)

In [ ]:
%timeit simpson_np(np.cos, X[0], X[-1], len(X))

In [ ]:
scipy.integrate.simps(np.cos(X), X) - simpson_np(np.cos, X[0], X[-1], len(X))

In [ ]:
scipy.integrate.simps?

# Your "calculus" module – build your own libraries

Aim: Collect your calculus functions in a python **module** for re-use.

* just a file *calculus.py* containing the code (create it with your editor)
* can be imported with `import calculus` (file must be in same directory)
* only add *functions*, *constants* (and *class* definitions) – avoid code that is immediately executed
* add comment doc string at top summarizing the module

## Problem: create the `calculus` module
Should contain
* integration functions `trapezoid` and `simpson` (numpy versions preferred)
* differentiation functions `D_fd` and `D_cd`

Testing:

In [ ]:
import calculus
print(np.allclose(calculus.simpson(np.cos, -10*np.pi, np.pi/2, 101), 1))
print(np.allclose(calculus.D_cd(np.cos, np.pi/2, 1e-6)), -1)